In [1]:
import pandas as pd

In [95]:
dataDir = '/gpfs/commons/groups/gursoy_lab/ajoglekar/Projects/2023_03_01_multiwayInteractions/2023_03_01_v0_dataGathering/PoreC_Fibroblasts/'
chromSizes = pd.read_csv(f'{dataDir}hg38.chromSizes',sep="\t", names = ['chr','size']).set_index('chr')['size'].to_dict()
readConcatemersWClosestGene = f'{dataDir}neonatal_fragsOutput_byChr/neonatal_fibroblasts_chr8.gz'
colnames = ["chr","start","end","readID","readLen","readQual",
"geneChr","geneStart","geneEnd","strand","geneID","bioType","geneName","dist","ID"]

fullBed = pd.read_csv(readConcatemersWClosestGene,sep = "\t",names = colnames)

In [96]:
chr19 = fullBed[fullBed['chr']=="chr8"]
binSize = 5*10**5
chrBins = [x for x in range(0,chromSizes['chr8']+binSize,binSize)]
chr19_binned = pd.cut(chr19['start'],bins = chrBins, labels = ["Bin"+str(i+1) for i in range(len(chrBins)-1)]).rename("binID")
chr19_wBinID = chr19.merge(chr19_binned,left_index=True,right_index=True)

In [97]:
chr19_wBinID[['ID','binID']].head()

,ID,binID
0,1,Bin1
1,2,Bin1
2,2,Bin1
3,2,Bin1
4,3,Bin1


In [98]:
groupedBins = chr19_wBinID.groupby('ID')['binID'].apply(list).reset_index(name='Bins')

In [99]:
def sort_key(item):
    return int(item.split('Bin')[1])

edges = ["_".join(sorted(list(set(a)), key=sort_key)) for a in groupedBins['Bins'] if len(list(set(a))) > 1]
readIDs = [groupedBins.iloc[ix][0] for ix in range(len(groupedBins)) if len(list(set(groupedBins.iloc[ix][1]))) > 1]
print(len(edges))
print(len(readIDs))

52843
52843


In [100]:
from collections import Counter
edgeDict = dict(Counter(edges))


In [101]:
edgeDict

{'Bin1_Bin50_Bin242': 1,
 'Bin1_Bin109': 2,
 'Bin5_Bin30': 4,
 'Bin6_Bin10': 4,
 'Bin6_Bin43': 2,
 'Bin7_Bin34': 2,
 'Bin7_Bin135': 2,
 'Bin7_Bin25_Bin52': 1,
 'Bin7_Bin196': 1,
 'Bin7_Bin8_Bin9_Bin80': 1,
 'Bin7_Bin24_Bin26_Bin32': 1,
 'Bin7_Bin9': 13,
 'Bin7_Bin56': 2,
 'Bin8_Bin31': 4,
 'Bin8_Bin14_Bin55': 1,
 'Bin9_Bin21': 1,
 'Bin9_Bin70': 1,
 'Bin9_Bin50': 2,
 'Bin10_Bin175': 1,
 'Bin10_Bin173': 1,
 'Bin10_Bin31_Bin65': 1,
 'Bin10_Bin11_Bin34_Bin35': 1,
 'Bin10_Bin13_Bin32_Bin36_Bin252': 1,
 'Bin10_Bin11_Bin14': 1,
 'Bin11_Bin253': 1,
 'Bin11_Bin213': 1,
 'Bin11_Bin137': 1,
 'Bin11_Bin149': 2,
 'Bin11_Bin15_Bin28': 1,
 'Bin11_Bin47_Bin50': 1,
 'Bin12_Bin101_Bin104': 1,
 'Bin12_Bin26': 2,
 'Bin13_Bin55': 1,
 'Bin13_Bin98_Bin184_Bin205': 1,
 'Bin13_Bin41_Bin63': 1,
 'Bin14_Bin151': 2,
 'Bin14_Bin129': 2,
 'Bin15_Bin29': 3,
 'Bin16_Bin25': 2,
 'Bin17_Bin115': 1,
 'Bin17_Bin24': 4,
 'Bin17_Bin71': 2,
 'Bin18_Bin283': 2,
 'Bin18_Bin34': 4,
 'Bin18_Bin45_Bin57_Bin62': 1,
 'Bin18_Bin19'

In [102]:
import pickle

with open(f'{dataDir}adult_chr8_dict.pkl','wb') as f:
    pickle.dump(edgeDict,f)

In [103]:
import pandas as pd
import numpy as np
import random
import statistics
from itertools import combinations
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import wasserstein_distance
from scipy.stats import energy_distance
import matplotlib.pyplot as plt
import seaborn as sns

class multiwayEval_realData:

    def __init__(self, keyCard, hpEdges, hpKeys, hpKeys_split, seed, 
                 toChoose,toPlotRef, toPlotInd, toPlotScatter,
                 quartile, plotDir,outDir):
        self.keyCard = keyCard
        self.hpEdges = hpEdges
        self.hpKeys = hpKeys
        self.hpKeys_split = hpKeys_split
        self.seed = seed
        self.toPlotRef = toPlotRef
        self.toPlotInd = toPlotInd
        self.toPlotScatter = toPlotScatter
        self.toChoose = toChoose
        self.qt = quartile
        self.plotDir = plotDir
        self.outDir = outDir

    def getReadSupportStatsPerCard(self, ixList, card):
        """Get disribution of read support per card so as to impose cutoffs
        in the end as to the trustworthiness of reads"""
        readSupps = [self.hpEdges[self.hpKeys[ix]] for ix in ixList]
        rsStats = pd.Series(readSupps).describe()
        rsStats.to_pickle(f'{self.outDir}readStats_card{card}.pkl')
        return

    def makeAllReferenceHashDicts(self):
        """For all available cards, create look up table of probability of
        seeing all possible subsets as a function of mean distances
        between subsets"""
        probHash = defaultdict(dict)

        for card in range(max(self.keyCard),2,-1):
            print("Calculating for card=",card)
            ixList = [index for index,element in enumerate(self.keyCard) if element == card]
            print("There are ",len(ixList),"reads")
            self.getReadSupportStatsPerCard(ixList,card)
            for n in range(2,card):
                print(f"Creating a probability hash for {n}-way subsets")
                hashID = f'{card}sub{n}'
                probHash[hashID] = self.getNWayProbsPerCard(card,n)
        return(probHash)
    
    def getNWayProbsPerCard(self,card,n):
        """Get distance-stratified expected probability distribution for high
        cardinality reads and their subsets of cardinality n"""
        ixList = [index for index,element in enumerate(self.keyCard) if element == card]
        mainDict = defaultdict(int)

        for ix in ixList:
            readSupport, nWayDict = self.makeNWayDict(ix,n)
            for key in nWayDict.keys():
                mainDict[key] += nWayDict[key]

        normalized_values = self.getProbabilitiesByDist(mainDict)

        if normalized_values is None: ##### TEST
            return 
        else:
            if self.toPlotRef is True:
                self.plotReadFreqsPerCard(mainDict,normalized_values,card,n)
            
            probabilityDict = {list(mainDict.keys())[ix]:normalized_values[ix] for ix in range(len(normalized_values))}
            return(probabilityDict)
    
    def makeNWayDict(self,ix,n): ### Breaks down when filtered
        """For a high cardinality read, make a dictionary
        that outputs the frequency and mean distance for subsets
        of cardinality n"""
        readSupport = self.hpEdges[self.hpKeys[ix]]
        splitKey = self.hpKeys_split[ix]
        combs = list(combinations(splitKey,n))
        subsetDict = defaultdict(int)
        for comb in combs:
            subsetEdge = '_'.join(map(str, comb))
            try:
                subsetEdgeReads = self.hpEdges[subsetEdge]
            except KeyError:
                subsetEdgeReads = 0
            meanDist = self.getNWayMeanDistPerSubset(comb)
            subsetDict[meanDist] = subsetEdgeReads
        return(readSupport, subsetDict)
    
    def getNWayMeanDistPerSubset(self,comb):
        """Get the mean pairwise distance given a
        high cardinality read"""
        twoWays = list(combinations(comb,2))
        twoWayDist = [self.getPairwiseDist(c) for c in twoWays]
        # meanDist = round(statistics.mean(twoWayDist))
        meanDist = round(statistics.geometric_mean(twoWayDist)) ## geometric mean
        return(meanDist)
    
    def getPairwiseDist(self,combination):
        """Given a two-way interaction, find dist between them"""
        ends = [int(combination[i].split("Bin")[1]) for i in [0,1]]
        diff = ends[1] - ends[0]
        return(diff)
    
    def getProbabilitiesByDist(self,mainDict):
        """Get probabilities of reads occurring by mean dist to
        obtain the expected probability distribution from the data"""
        totalReadsForCard = sum(mainDict.values())
        if totalReadsForCard == 0:
            return None
        else:
            normalized_values = [value / totalReadsForCard for value in mainDict.values()]
            ## return total reads as well
            return(normalized_values)
    

    def plotReadFreqsPerCard(self,mainDict,normalized_values,card,n):
        """Takes in a generated dictionary and plots the
        read frequency as well as probabilities (first 20) by the
        distance"""

        # Step 1: Plot a barplot with dictionary keys on the x-axis and values on the y-axis
        sns.barplot(x=list(mainDict.keys()), y=list(mainDict.values()),color = "grey")
        plt.title(f"Read frequency of {n}-way subsets for Card={card}")
        plt.xlabel(f"Mean (Geom) distance between {n}-way interactions")
        plt.ylabel("Frequency")
        plt.xticks(rotation=90,fontsize=5)
        plt.savefig(f'{self.plotDir}DistStratProbs_Card{card}sub{n}.png',bbox_inches = 'tight',facecolor = "white")
        #plt.show()

        # Step 2: Plot the same barplot with the ratio of each value to the total
        sns.barplot(x=list(mainDict.keys()), y=normalized_values,color = "grey")
        plt.title(f"Probability of occurrence {n}-way distance for Card={card}")
        plt.xlabel(f"Mean (Geom) distance between {n}-way interactions")
        plt.ylabel("Probabilities")
        plt.xlim(0,19)
        plt.xticks(rotation=90)
        plt.savefig(f'{self.plotDir}DistStratProbs_top20_Card{card}sub{n}.png',bbox_inches = 'tight',facecolor = "white")
        #plt.show()
        plt.close()
        return None
    
    def statsForAllReads(self,probHash):
        """Wrapper for all reads"""
        for card in range(max(self.keyCard),3,-1):
            allStats = self.statsForAllCardSubsets(card,probHash)
            self.processConcatDFs(allStats[0],"wDist",False,card)
            self.processConcatDFs(allStats[1],"cosineSim",True,card)
            self.processConcatDFs(allStats[2],"eDist",False,card)
            self.processConcatDFs(allStats[3],"empDist",True,card)
        for card in [3]:
            allStats = self.statsForAllCardSubsets(card,probHash)
            self.processConcatDFs(allStats[0],"wDist",False,card)
            self.processConcatDFs(allStats[1],"cosineSim",True,card)
            self.processConcatDFs(allStats[2],"eDist",False,card)
            self.processConcatDFs(allStats[3],"empDist",True,card)            
        return None

    def processConcatDFs(self,metricDF,metricName,simiBool,card):
        """summarize and write output for each metric"""
        if simiBool is True:
            qtCutoff = self.qt
            operator = "leq"
        else:
            qtCutoff = 100 - self.qt
            operator = "geq"
        if card == 3:
            metricCutoff = pd.Series(metricDF['3Sub2']).describe()[f'{qtCutoff}%']
            self.plotSimilarityHist(metricDF['3Sub2'],metricName,card)
            if operator == "leq":
                mStatus = [1 if x else 0 for x in (metricDF['3Sub2'] <= metricCutoff)]
            else:
                mStatus = [1 if x else 0 for x in (metricDF['3Sub2'] >= metricCutoff)]
        else:
            summaryMetric = metricDF.filter(like="Sub").agg((np.mean,np.std),axis = 1)        
            metricCutoff = self.getCutoff(summaryMetric,qtCutoff)
            self.plotSimilarityHist(summaryMetric['mean'],metricName,card)
            if operator == "leq":
                mStatus = [1 if x else 0 for x in (summaryMetric['mean'] <= metricCutoff)]
            else:
                mStatus = [1 if x else 0 for x in (summaryMetric['mean'] >= metricCutoff)]
            if self.toPlotScatter is True:
                self.plotScatterWithErrorBars(summaryMetric,metricName,card)
        metricDF['Status'] = mStatus
        print(f"Writing output for {metricName}")
        metricDF.to_csv(f'{self.outDir}/{metricName}_card{card}.csv',sep = "\t",index=False)
        return None
    
    def getCutoff(self,summaryDF,quartile):
        q = f'{quartile}%'
        cutoff = pd.Series(summaryDF['mean']).describe()[q]
        return(cutoff)

    def statsForAllCardSubsets(self,card,probHash):
        """Wrapper for the stats per card. Calculates for all subsets of a card and binds into a df"""
        print("Calculating for card=",card)
        ixList = [index for index,element in enumerate(self.keyCard) if element == card]
        print("There are ",len(ixList),"reads")
        cardToChoose = min(self.toChoose,len(ixList))
        print("Calculating stats for",cardToChoose,"reads")
        random.seed(self.seed)
        revised_ixes = random.sample(ixList,cardToChoose)
        C1 = []
        C2 = []
        C3 = []
        C4 = []
        for n in range(2,card):
            stats = self.getStatsPerCard(card,n,probHash,revised_ixes)
            C1.append(stats[0])
            C2.append(stats[1])
            C3.append(stats[2])
            C4.append(stats[3])
            if n == 2:
                C5 = stats[4]
        cN = [str(card)+"Sub"+str(i) for i in range(2,card)]
        df1 = pd.DataFrame(C1).T
        df2 = pd.DataFrame(C2).T
        df3 = pd.DataFrame(C3).T
        df4 = pd.DataFrame(C4).T
        df1.columns = cN
        df1['Edge_ix'] = revised_ixes
        df1['ReadSupport'] = C5
        df2.columns = cN
        df2['Edge_ix'] = revised_ixes
        df2['ReadSupport'] = C5
        df3.columns = cN
        df3['Edge_ix'] = revised_ixes
        df3['ReadSupport'] = C5
        df4.columns = cN
        df4['Edge_ix'] = revised_ixes
        df4['ReadSupport'] = C5
        return(df1,df2,df3,df4)

    def getStatsPerCard(self,card,n,probHash,revised_ixes):
        """Per cardinality, get a subset of reads (for computational
        purposes), calculate the similarity of observed n-way interactions
        to the expected value, and output a list"""
        wdistList = []
        cosList = []
        edistList = []
        empDistList = []
        readSuppList = []
        expHash = f'{card}sub{n}'
        for ix in revised_ixes:
            stats = self.getReadExpectednessStats(card,ix,n,probHash[expHash])
            if stats is not None:
                wdistList.append(stats[0])
                cosList.append(stats[1])
                edistList.append(stats[2])
                empDistList.append(stats[3])
                readSuppList.append(stats[4])
        return(wdistList, cosList, edistList, empDistList, readSuppList)
    
    def getReadExpectednessStats(self,card,ix,n,hash):
        """Per read, get the observed distribution of n-way contacts
        and calculate similarity to the expected distribution"""
        readSupport, readDict = self.makeNWayDict(ix,n)
        readPercs= self.getProbabilitiesByDist(readDict)
        if readPercs is None:
            return None
        else:
            probVals = [hash[k] for k in readDict.keys()]
            normProbs = [p / sum(probVals) for p in probVals]
            probVals = normProbs  ## Normalizing mostly for the empirical calculation
            if self.toPlotInd is True:
                if len(readPercs) > 2 and len(probVals) > 2:
                    self.makeSanityCheckPlotsPerRead(readDict,readPercs,probVals,card,n,ix)
            wDist = wasserstein_distance(readPercs, probVals)
            similarity = cosine_similarity(np.array(readPercs).reshape(1,-1), 
                                        np.array(probVals).reshape(1,-1))[0,0]
            eDist = energy_distance(readPercs, probVals)
            empDist = self.calcEmpDist(readPercs, probVals)
            return(wDist, similarity, eDist, empDist, readSupport)
    
    def calcEmpDist(self,readPercs, probVals):
        """Trying a different metric for distance. 
        We will figure out the cutoff later"""
        obs = np.log(readPercs)
        exp = np.log(probVals)
        if 0 in exp:
            meanOE = 1
        else:
            obsOverExp = obs/exp
            meanOE = np.mean(obsOverExp[np.isfinite(obsOverExp)])
        return(meanOE)

    def makeSanityCheckPlotsPerRead(self,readDict, readPercs, probVals, card, n, ix):
        """For specific reads, plot the observed versus expected distributions
        of two-way interactions along with a fitted spline. Additionally outputs
        the slope (useful only if line) and wDist values"""
        Distances = list(readDict.keys())
        print(card,"sub",n)
        print(ix)
        #print(readPercs)
        #print(probVals)
        
        # Create a 2x2 grid of subplots
        fig, axs = plt.subplots(2, 2, figsize=(6, 6))
        # Plot 1: Observed w/ spline
        lowess1 = sns.regplot(x=Distances, y=readPercs, 
                              lowess=True, ci=None, color='red', ax=axs[0, 0])
        axs[0, 0].set_title(f"Observed w/ spline Card{card}sub{n}")
        # Plot 2: Expected w/ spline
        lowess2 = sns.regplot(x=Distances, y=probVals, 
                              lowess=True, ci=None, color='grey', ax=axs[0, 1])
        axs[0, 1].set_title(f"Expected w/ spline Card{card}sub{n}")

        # Calculate smoothed values and slopes
        y_smoothed1 = lowess1.get_lines()[0].get_ydata()
        y_smoothed2 = lowess2.get_lines()[0].get_ydata()
        slope1 = np.gradient(y_smoothed1)
        slope2 = np.gradient(y_smoothed2)

        # Plot 3: Barplot for readPercs
        sns.barplot(x=list(readDict.keys()), y=readPercs, ax=axs[1, 0])
        axs[1, 0].set_title(f"Barplot for readPercs Card{card}sub{n}")
        # Plot 4: Barplot for probVals
        sns.barplot(x=list(readDict.keys()), y=probVals, ax=axs[1, 1])
        axs[1, 1].set_title(f"Barplot for normalized prior probs Card{card}sub{n}")

        #correlation = np.corrcoef(readPercs, probVals)[0, 1]
        wDist = wasserstein_distance(readPercs, probVals)
        similarity = cosine_similarity(np.array(readPercs).reshape(1,-1), 
                    np.array(probVals).reshape(1,-1))[0,0]
        eDist = energy_distance(readPercs, probVals)
        empDist = self.calcEmpDist(readPercs, probVals)
        
        # Print the slopes
        print("Comparing -------------")
        print(f"Slope for observed: {slope1.mean()}")
        print(f"Slope for expected: {slope2.mean()}")
        print(f"Wasserstein distance: {wDist}")
        print(f"Energy distance: {eDist}")
        print(f"Empirical distance: {empDist}")
        print(f"Cosine similarity: {similarity}")

        # Adjust layout and show the subplots
        plt.tight_layout()
        plt.savefig(f'{self.plotDir}/SingleReadPlot_Card{card}sub{n}_{ix}.png',bbox_inches = 'tight',facecolor = "white")
        plt.show()
        plt.close()
        return None

    def plotSimilarityHist(self,metricDistr,metricName,card):
        """Given distribution for a cardinality, plot the wass dist
        and cosine similarity values so that we can settle on a heuristic"""
        plt.figure(figsize=(4,4))
        plt.hist(metricDistr,color='blue', alpha=0.7, width=0.3,bins = 201)
        plt.xlim(-1,1.01)
        plt.title(f"Mean {metricName} distance for card={card}")
        plt.close()
        return None

    def plotScatterWithErrorBars(self,summaryDF,metric,card):
        """Given summary stats for a certain number of reads, plot scatter plot with error bars 
        representing change over all subsets"""
        sortedSumm = summaryDF.sort_values(by='mean')
        x = [i+1 for i in range(summaryDF.shape[0])] # Use the index as x-axis
        y = sortedSumm['mean']
        error = sortedSumm['std']

        plt.scatter(x, y, label=f'mean {metric}', marker='o')
        plt.errorbar(x, y, yerr=error, linestyle='None', color='grey', capsize=3)
        plt.xlabel('ReadID')
        plt.ylabel(f'mean {metric}')
        plt.ylim(0,2)
        plt.title(f'Distribution for card={card}')
        plt.legend()
        plt.savefig(f'{self.plotDir}/ScatterPlot_{metric}_Card{card}_max{self.toChoose}reads.png',
                    bbox_inches = 'tight',facecolor = "white")
        #plt.show()
        plt.close()
        return None


In [104]:
dataDir = '/gpfs/commons/groups/gursoy_lab/ajoglekar/Projects/2023_03_01_multiwayInteractions/2023_03_01_v0_dataGathering/PoreC_Fibroblasts/'
runDir = 'v1.evaluateExpectedVersusInteresting_adult/'
plotDir = f'{dataDir}{runDir}Plots_adult_chr8/'
outDir = f'{dataDir}{runDir}dfs_adult_chr8/'
inputPkl = 'adult_chr8_dict.pkl'
probHashOutName = f'{outDir}probHash_chr8.json'

seed = 1
quartile = 25
toChoose = 400
toPlotRef = False
toPlotInd = False
toPlotScatter = False

In [106]:
import os
import json
import warnings

print("Loading in hyperedge pickle file")
with open(f'{dataDir}/{inputPkl}','rb') as f:
    hpEdges = pickle.load(f)

print("Processing all the hyperedges from pickle file")
hpKeys = [k for k in hpEdges.keys()]
print("A total of",len(hpKeys),"initial interactions")

readSupport = [v for v in hpEdges.values()]
atLeastTwoChains = [i for i,x in enumerate(readSupport) if x >=2]
updatedDict = {hpKeys[i]:readSupport[i] for i in atLeastTwoChains}

hpKeys = [k for k in updatedDict.keys()]
hpKeys_split = [k.split("_") for k in updatedDict.keys()]
keyCard = [len(item) for item in hpKeys_split]

print("Updated the input: retaining",len(hpKeys),
        "interactions with chain support of at least 2")

evalInstance = multiwayEval_realData(keyCard, updatedDict, hpKeys, hpKeys_split, seed,
                            toChoose, toPlotRef, toPlotInd, toPlotScatter,
                            quartile, plotDir,outDir)

if os.path.exists(probHashOutName):
    print("Expected probabilities file already exists...moving on")
    with open(probHashOutName,'r') as file:
        tmpHash = json.load(file)
        probHash = {key: {int(k): v for k, v in value.items()} 
                    for key, value in tmpHash.items()}
else:
    print("Expected probabilities file does not exist...creating now")
    probHash = evalInstance.makeAllReferenceHashDicts()
    with open(probHashOutName,'w') as file:
        json.dump(probHash,file)
    print("File created...moving on")

warnings.filterwarnings('ignore')


Loading in hyperedge pickle file
Processing all the hyperedges from pickle file
A total of 36598 initial interactions
Updated the input: retaining 7526 interactions with chain support of at least 2
Expected probabilities file does not exist...creating now
Calculating for card= 7
There are  1 reads
Creating a probability hash for 2-way subsets
Creating a probability hash for 3-way subsets
Creating a probability hash for 4-way subsets
Creating a probability hash for 5-way subsets
Creating a probability hash for 6-way subsets
Calculating for card= 6
There are  1 reads
Creating a probability hash for 2-way subsets
Creating a probability hash for 3-way subsets
Creating a probability hash for 4-way subsets
Creating a probability hash for 5-way subsets
Calculating for card= 5
There are  4 reads
Creating a probability hash for 2-way subsets
Creating a probability hash for 3-way subsets
Creating a probability hash for 4-way subsets
Calculating for card= 4
There are  26 reads
Creating a probabil

In [107]:
toChoose = 400

evalInstance = multiwayEval_realData(keyCard, updatedDict, hpKeys, hpKeys_split, seed,
                            toChoose, toPlotRef, toPlotInd, toPlotScatter,
                            quartile, plotDir,outDir)

A = evalInstance.statsForAllReads(probHash)

Calculating for card= 7
There are  1 reads
Calculating stats for 1 reads
Writing output for wDist
Writing output for cosineSim
Writing output for eDist
Writing output for empDist
Calculating for card= 6
There are  1 reads
Calculating stats for 1 reads
Writing output for wDist
Writing output for cosineSim
Writing output for eDist
Writing output for empDist
Calculating for card= 5
There are  4 reads
Calculating stats for 4 reads


ValueError: Length of values (4) does not match length of index (3)